In [2]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
companies = '''
Apple
Airbnb
Twitch
Servicenow
Stripe
Slack
Okta, inc.
Microsoft
Tesla
Instacart
Square
Netapp
Visa
Affirm, Inc.
Paypal
Autodesk
Quora
Snap Inc.
Workday
Opendoor.Com
Cerner Corporation
Yelp
Zillow
Sofi
Roblox
'''

In [4]:
companies.strip('\n').split('\n')

['Apple',
 'Airbnb',
 'Twitch',
 'Servicenow',
 'Stripe',
 'Slack',
 'Okta, inc.',
 'Microsoft',
 'Tesla',
 'Instacart',
 'Square',
 'Netapp',
 'Visa',
 'Affirm, Inc.',
 'Paypal',
 'Autodesk',
 'Quora',
 'Snap Inc.',
 'Workday',
 'Opendoor.Com',
 'Cerner Corporation',
 'Yelp',
 'Zillow',
 'Sofi',
 'Roblox']

In [5]:
gh_urls = {
    # 'Faire': 'https://boards.greenhouse.io/faire',
    'Point72': 'https://boards.greenhouse.io/point72',
    'Twitch': 'https://boards.greenhouse.io/twitch',
    # 'Faire': 'https://boards.greenhouse.io/embed/job_board?for=airbnb',
    'Ogilvy Health USA': 'https://boards.greenhouse.io/ogilvyhealthusa',
    'Ogilvy UK': 'https://boards.greenhouse.io/ogilvyuk'
}
    

In [1]:
import time
import requests
import json
from bs4 import BeautifulSoup

def load_page(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 ' + \
        '(KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
    response = requests.get(url, headers={'User-Agent': user_agent})
    return response


def parse_greenhouse_page(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    jobs = []
    for div in soup.find_all('div', {'class':'opening'}):
        job_title = div.find('a').text
        job_url = 'https://boards.greenhouse.io' + div.find('a')['href']
        job_loc = div.find('span').text
        element = div
        hierarchy = []
        while element and (element.name != 'section' or 'level-0' not in element.get('class', [])):
            element = element.parent
            hierarchy.append(element.find_all()[0].get_text(separator=' '))
        jobs.append({
            'Title': job_title,
            'URL': job_url,
            'Location': job_loc,
            'Hierarchy': hierarchy
        })
    return jobs

def fwprint(*things, w=14, **kwargs):
    trunc_things = [str(thing)[:w] for thing in things]
    fw_things = [thing+(' '*(w-len(thing))) for thing in trunc_things]
    print(*fw_things, **kwargs)
    
def save_to_db(company_name, jobs, scraped_at):
    print()
    fwprint('Title', 'Company', 'Location', 'Department', 'Scraped_at', 'URL', sep=' | ')
    fwprint(*['-'*12]*6, sep=' | ')
    for job in jobs:
        title = job['Title']
        location = job['Location']
        department = job['Hierarchy']
        url = job['URL']
        
        fwprint(title, company_name, location, department, scraped_at, url, sep=' | ')
    print()


def update_status(timestamp, id, task, scraper_type, company, site, url):
    fwprint(timestamp, '01', task, scraper_type, company, site, url, sep=' | ')


def scrape_company_page(company_name, company_page_url, jobs_site='Greenhouse'):
    fwprint('timestamp', 'id', 'task', 'scraper_type', 'company', 'site', 'url', sep=' | ')
    fwprint(*['-'*12]*7, sep=' | ')

    scraped_at = time.time()
    update_status(
        timestamp=scraped_at,
        id='01',
        task='Scraping page',
        scraper_type='company',
        company=company_name,
        site=jobs_site,
        url=company_page_url
    ) # initial task
    response = load_page(company_page_url)

    parsed_at = time.time()
    update_status(
        timestamp=parsed_at,
        id='01',
        task='Parsing page',
        scraper_type='company',
        company=company_name,
        site=jobs_site,
        url=company_page_url
    ) # parsing
    if jobs_site=='Greenhouse':
        jobs = parse_greenhouse_page(response)

    stored_at = time.time()
    update_status(
        timestamp=stored_at,
        id='01',
        task='Storing jobs in DB',
        scraper_type='company',
        company=company_name,
        site=jobs_site,
        url=company_page_url
    ) # saving to db
    save_to_db(company_name, jobs, scraped_at)

    finished_at = time.time()
    update_status(
        timestamp=finished_at,
        id='01',
        task='Done',
        scraper_type='company',
        company=company_name,
        site=jobs_site,
        url=company_page_url
    ) # done


if __name__ == "__main__":
    scrape_company_page('Twitch', 'https://boards.greenhouse.io/twitch')

SyntaxError: invalid syntax (3002110595.py, line 109)

In [ ]:
for company_name, company_page_url in gh_urls.items():
    scrape_company_page(company_name, company_page_url)

In [53]:
print('hi', 'there', 'hello', sep='\t')

hi	there	hello


In [ ]:
7

In [55]:
fwprint('hi', 'there', 'hello', sep='\t')

hi          	there       	hello       


In [38]:
import requests

In [48]:
url = 'https://0ozmeqv5h5.execute-api.us-east-1.amazonaws.com/default/hey-lambda?paroooometer=Sargam&param2=Kevin,joe'

In [49]:
response = requests.get(url)#, headers=headers)

In [50]:
response.text

"{'param2': 'Kevin,joe', 'paroooometer': 'Sargam'}"

In [ ]:
url = 'https://0ozmeqv5h5.execute-api.us-east-1.amazonaws.com/live/scrape-company-jobs-page'
params={key: value}

?site=greenhouse&url=https://boards.greenhouse.io/point72'

response = requests.get(url, params=params)
response.text

In [ ]:
orchestrator:

workers come to the orch for direction

orch spins up workers
orch makes a note of which worker came to it and what task it gave to that worker:

    SCRAPER_STATUS
        timestamp   | id  | task                   | sc_type | company | site | url
        ==============================================================================================================================
        69865238946 | 03  | started scraping       | company | point72 | gnhs | https://boards.greenhouse.io/point72
        69524397867 | 01  | finished scraping      | company | ogilvy  | gnhs | https://boards.greenhouse.io/ogilvyuk
        69865238946 | 02  | pushing jobs to db     | company | meta    | -    | -
        69865238946 | 02  | pushed jobs to db      | company | meta    | -    | -
        # 57824968598 | 05  | idle                   | company | -       | -    | -
        69865238946 | 103 | started scraping       | job     | point72 | gnhs | https://boards.greenhouse.io/ogilvyuk/jobs/4298945005
        69524397867 | 101 | finished scraping      | job     | ogilvy  | gnhs | https://boards.greenhouse.io/ogilvyuk/jobs/1698943094
        69865238946 | 102 | pushing job data to db | job     | meta    | gnhs | https://boards.greenhouse.io/ogilvyuk/jobs/5326623322
        69865238946 | 102 | pushed job data to db  | job     | meta    | gnhs | https://boards.greenhouse.io/ogilvyuk/jobs/7534352334

they leave with a task - status scraping
they check back in when they finish parsing
then they store the data in the bd
and say when they're done with that
then they're idle
and then they ask for more work
